In [9]:
import pandas as pd

In [39]:
# 빅카인즈에서 내려받은 엑셀 업로드
df = pd.read_csv('./성범죄_0826_0902.csv')

In [41]:
# 제목만 따오기
title_list=df['제목']
title_list

0      ‘미성년 11명 성폭행’ 김근식 내달 출소 법무부 “1대 1 전자감독 대상”
1                     [단독] 교사도, 강사도 ‘엘’은 어디에나 있었다
2                 미성년 12명 성폭행 김근식, 10월 출소 불안한 주민들
3                  "야채만 먹였다" 18개월 아들 영양실조로 죽게한 엄마
4                 미성년 11명 유린 성폭행범 출소 "전자발찌 채운다는데"
                          ...                    
470          “명백한 동의 없다면 성폭행” 스페인 ‘비동의 강간죄’ 만들어진다
471                                     [분수대] 음성권
472         미성년자 성매매 강요 영리 취득 검찰, 조직 1명에 징역 7년 구형
473            성추행 갑질논란 체육회 전간부, 벌금형 40시간 치료교육 명령
474              [꿈꾸는 경기교육] 낙태가 아닌 임신 중단으로 불리는 그날
Name: 제목, Length: 475, dtype: object

# 네이버 댓글 url 가져오기

In [44]:
from selenium import webdriver
import time

driver = webdriver.Chrome('./chromedriver')
driver.implicitly_wait(2)

In [83]:
# 매크로 방지 피하기 위해 wait을 걸어줍니다
driver.implicitly_wait(2)

# url 넣을 리스트 생성
url_list = []

for title in title_list[408:]:
    try:
        url_format = 'https://search.naver.com/search.naver?where=news&query={}&sm=tab_dgs&qdt=0'
        # 검색 결과 창 띄우기
        driver.get(url_format.format(title))
        time.sleep(2)

        # 기사 섹션 접근
        list_news = driver.find_element_by_class_name('group_news')
        li = list_news.find_element_by_tag_name('li')

        # '네이버뉴스' 클릭할 수 있는 상단 섹션 접근
        info_group = li.find_element_by_class_name('info_group')

        # 모든 url 가져오기
        a_s = info_group.find_elements_by_tag_name('a')

        # url 중 네이버뉴스에 해당하는 url만 리스트에 추가
        for a in a_s:
            url = a.get_attribute('href')

            if 'naver' in url:
                url_list.append(url)
    except:
        continue

In [93]:
# 스포츠뉴스 url은 형식이 달라서 삭제 처리    
url_list = [s for s in url_list if "sports" not in s] 

460


In [96]:
url_list

['https://n.news.naver.com/mnews/article/022/0003731310?sid=102',
 'https://n.news.naver.com/mnews/article/056/0011332604?sid=102',
 'https://n.news.naver.com/mnews/article/629/0000170357?sid=102',
 'https://n.news.naver.com/mnews/article/011/0004095048?sid=104',
 'https://n.news.naver.com/mnews/article/011/0004095042?sid=102',
 'https://n.news.naver.com/mnews/article/025/0003221246?sid=102',
 'https://n.news.naver.com/mnews/article/052/0001784684?sid=102',
 'https://n.news.naver.com/mnews/article/025/0003221245?sid=100',
 'https://n.news.naver.com/mnews/article/015/0004745387?sid=102',
 'https://n.news.naver.com/mnews/article/022/0003731274?sid=102',
 'https://n.news.naver.com/mnews/article/277/0005142239?sid=102',
 'https://n.news.naver.com/mnews/article/015/0004745282?sid=104',
 'https://n.news.naver.com/mnews/article/052/0001784622?sid=100',
 'https://n.news.naver.com/mnews/article/469/0000695001?sid=102',
 'https://n.news.naver.com/mnews/article/082/0001172671?sid=102',
 'https://

# 댓글 url에서 댓글 크롤링

- 누가 짜놓은거 그대로 갖고 왓습니다,,

In [99]:
# 댓글을 달 빈 리스트를 생성합니다.
List = []
# 라이브러리를 로드합니다.
from bs4 import BeautifulSoup
import requests
import re
import sys
import pprint
 
# 네이버 뉴스 url을 입력합니다.
for url in url_list:
    oid = url.split('/')[-2]
    aid = url.split('/')[-1].split("?")[0]

    page = 1
    header = {
        "User-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/65.0.3325.181 Safari/537.36",
        "referer": url,
    }

    while True:
        c_url = "https://apis.naver.com/commentBox/cbox/web_neo_list_jsonp.json?ticket=news&templateId=default_society&pool=cbox5&_callback=jQuery1707138182064460843_1523512042464&lang=ko&country=&objectId=news" + oid + "%2C" + aid + "&categoryId=&pageSize=20&indexSize=10&groupId=&listType=OBJECT&pageType=more&page=" + str(
            page) + "&refresh=false&sort=FAVORITE"
        # 파싱하는 단계입니다.
        r = requests.get(c_url, headers=header)
        cont = BeautifulSoup(r.content, "html.parser")
        total_comm = str(cont).split('comment":')[1].split(",")[0]

        match = re.findall('"contents":([^\*]*),"userIdNo"', str(cont))
        # 댓글을 리스트에 중첩합니다.
        List.append(match)
        # 한번에 댓글이 20개씩 보이기 때문에 한 페이지씩 몽땅 댓글을 긁어 옵니다.
        if int(total_comm) <= ((page) * 20):
            break
        else:
            page += 1
 
 
# 여러 리스트들을 하나로 묶어 주는 함수입니다.
def flatten(l):
    flatList = []
    for elem in l:
        # if an element of a list is a list
        # iterate over this list and add elements to flatList
        if type(elem) == list:
            for e in elem:
                flatList.append(e)
        else:
            flatList.append(elem)
    return flatList
 
 
# 리스트 결과입니다.
 
allCommetns = flatten(List) 
print(len(allCommetns))

60046


In [100]:
# 댓글 앞뒤로 "" 붙어있는거 신경쓰여서 잘랐습니다
comment_list = []
for comment in allCommetns:
    comment_list.append(comment[1:-1])

# 데이터프레임 추출
df = pd.DataFrame([comment_list]).transpose()

In [110]:
# 댓글이 6만개가 나와서 ... 어디선가 중복된 것 같아서 중복 제거 했습니다...
df.drop_duplicates(subset=None, keep='first', inplace=True, ignore_index=True)

In [116]:
pip install openpyxl

Note: you may need to restart the kernel to use updated packages.


'C:\Users\SUYOUNG'은(는) 내부 또는 외부 명령, 실행할 수 있는 프로그램, 또는
배치 파일이 아닙니다.


In [117]:
# 댓글 4465개 엑셀 저장
df.to_excel('./성범죄_댓글.xlsx',index=False)